# Доверительные интервалы

In [1]:
import numpy as np
import pandas as pd

import scipy
from statsmodels.stats.weightstats import *
from statsmodels.stats.proportion import proportion_confint

Давайте уточним правило трёх сигм. Утверждение: 99.7% вероятностной массы случайной величины $ X∼N(μ,σ2)X\sim N\left(\mu,\sigma^2\right)X∼N(μ,σ2)$ лежит в интервале $ μ±c⋅σ\mu\pm c \cdot \sigmaμ±c⋅σ $. Чему равно точное значение константы ccc? Округлите ответ до четырёх знаков после десятичной точки. 

In [2]:
round(scipy.stats.norm.ppf(1 - 0.003 / 2.), 4)

2.9677

В пятилетнем рандомизированном исследовании Гарвардской медицинской школы 11037 испытуемых через день принимали аспирин, а ещё 11034 — плацебо. Исследование было слепым, то есть, испытуемые не знали, что именно они принимают.

За 5 лет инфаркт случился у 104 испытуемых, принимавших аспирин, и у 189 принимавших плацебо.

Оцените, насколько вероятность инфаркта снижается при приёме аспирина. Округлите ответ до четырёх знаков после десятичной точки.

In [3]:
aspirin_data = np.array( [1 if i<104 else 0 for i in range(11037)] )
placebo_data = np.array( [1 if i<189 else 0 for i in range(11034)] )

In [4]:
prob_infarction_aspirin = aspirin_data.sum() / aspirin_data.shape[0]
prob_infarction_placebo = placebo_data.sum() / placebo_data.shape[0]
print('Вероятность инфаркта миокарда (aspirin): %.4f' % prob_infarction_aspirin)
print('Вероятность инфаркта миокарда (placebo): %.4f' % prob_infarction_placebo)
print('Снижение вероятности инфаркта миокарда: %.4f' % (prob_infarction_placebo - prob_infarction_aspirin))

Вероятность инфаркта миокарда (aspirin): 0.0094
Вероятность инфаркта миокарда (placebo): 0.0171
Снижение вероятности инфаркта миокарда: 0.0077


Постройте теперь 95% доверительный интервал для снижения вероятности инфаркта при приёме аспирина. Чему равна его верхняя граница? Округлите ответ до четырёх знаков после десятичной точки.

In [5]:
def proportions_confint_diff_ind(p1, p2, alpha = 0.05):    
    z = scipy.stats.norm.ppf(1 - alpha / 2.)   
       
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ (11034.) + p2 * (1 - p2)/ (11037.))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ (11034.) + p2 * (1 - p2)/ (11037.))
    
    return (left_boundary, right_boundary)

In [6]:
X = proportions_confint_diff_ind((189./11034.), (104./11037.))

In [7]:
X

(0.004687750675049439, 0.010724297276960124)

In [8]:
round(X[1], 4)

0.0107

Продолжим анализировать данные эксперимента Гарвардской медицинской школы.

Для бернуллиевских случайных величин X∼Ber(p) часто вычисляют величину p1−p, которая называется шансами (odds). Чтобы оценить шансы по выборке, вместо p нужно подставить p^. Например, шансы инфаркта в контрольной группе, принимавшей плацебо, можно оценить как

Оцените, во сколько раз понижаются шансы инфаркта при регулярном приёме аспирина. Округлите ответ до четырёх знаков после десятичной точки.


In [9]:
def odds(data):
    p = data.sum() / data.shape[0]
    return p / (1 - p)

In [10]:
odds_aspirin = odds(aspirin_data)
print('Шансы (aspirin): %.4f' % odds_aspirin)
odds_placebo = odds(placebo_data)
print('Шансы (placebo): %.4f' % odds_placebo)

Шансы (aspirin): 0.0095
Шансы (placebo): 0.0174


In [11]:
print('Понижение шансов при приёме аспирина: %.4f' % (odds_placebo / odds_aspirin))

Понижение шансов при приёме аспирина: 1.8321


Величина, которую вы оценили в предыдущем вопросе, называется отношением шансов. Постройте для отношения шансов 95% доверительный интервал с помощью бутстрепа. Чему равна его нижняя граница? Округлите ответ до 4 знаков после десятичной точки.

Чтобы получить в точности такой же доверительный интервал, как у нас:

составьте векторы исходов в контрольной и тестовой выборках так, чтобы в начале шли все единицы, а потом все нули;

установите random seed=0;

сделайте по 1000 псевдовыборок из каждой группы пациентов с помощью функции get_bootstrap_samples.


In [12]:
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

In [13]:
def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

In [14]:
np.random.seed(0)
odds_aspirin_data = np.array(list(map(odds, get_bootstrap_samples(aspirin_data, 1000))))
odds_placebo_data = np.array(list(map(odds, get_bootstrap_samples(placebo_data, 1000))))

In [15]:
print('95%% доверительный интервал для уменьшения инфаркта: %s' %
      str(stat_intervals(odds_placebo_data / odds_aspirin_data, 0.05)))

95% доверительный интервал для уменьшения инфаркта: [1.44419465 2.34321168]
